In [1]:
import sys, os, csv, lucene
import numpy as np

In [2]:
from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.index import DirectoryReader, IndexWriter, IndexWriterConfig
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.document import Document, Field, FieldType, StringField, TextField
from org.apache.lucene.store import SimpleFSDirectory, RAMDirectory
from org.apache.lucene.util import Version

In [3]:
lucene.initVM(vmargs=['-Djava.awt.headless=true'])

In [4]:
fsDir = SimpleFSDirectory(File("index"))
reader = DirectoryReader.open(fsDir)
searcher = IndexSearcher(reader)

In [5]:
analyzer = StandardAnalyzer(Version.LUCENE_CURRENT)
parser = QueryParser(Version.LUCENE_CURRENT, "question", analyzer)

In [6]:
query = parser.parse('When athletes begin to exercise, their heart rates and respiration rates increase.  At what level of organization does the human body coordinate these functions?')

In [7]:
result = searcher.search(query, 10)
print result.getMaxScore()
scoreDocs = result.scoreDocs
for scoreDoc in scoreDocs:
    doc = searcher.doc(scoreDoc.doc)
    print [doc.get("question"), doc.get("answer"), scoreDoc.score]

6.00224876404
[u'When athletes begin to exercise, their heart rates and respiration rates increase. At what level of organization does the human body coordinate these functions?', u'at the system level', 6.002248764038086]
[u'At what level of organization does life begin?', u'Cells', 0.7390643954277039]
[u'at what level of organization does life begin', u'cell', 0.7390643954277039]
[u'An increase in infflation does what to interest rates', u'Higher inflation = higher interest rates', 0.7244764566421509]
[u'Level of Organization in the Human Body', u'Cell > Tissue > Organs > Organ Systems> Organisms', 0.671397864818573]
[u'What is the largest level of organization in the human body?', u'the largest level of organization in the human body is the organ system.', 0.6668251752853394]
[u'Birth rates rapidly decline,death rates continue to decline,and natural increase rates begin to moderate.', u'Stage 3: decreasing growth', 0.6202281713485718]
[u'Birth rates rapidly decline, death rates cont

In [8]:
# braindead way how to score similarity of two answers, but I didn't know Lucene any better
def score_answer(answer, answerA):
    directory = RAMDirectory()
    analyzer = StandardAnalyzer(Version.LUCENE_CURRENT)
    #analyzer = LimitTokenCountAnalyzer(analyzer, 10000)
    config = IndexWriterConfig(Version.LUCENE_CURRENT, analyzer)
    writer = IndexWriter(directory, config)

    doc = Document()
    doc.add(Field("answer", answer, TextField.TYPE_STORED))
    writer.addDocument(doc)

    writer.commit()
    writer.close()
    
    reader = DirectoryReader.open(directory)
    searcher = IndexSearcher(reader)

    analyzer = StandardAnalyzer(Version.LUCENE_CURRENT)
    parser = QueryParser(Version.LUCENE_CURRENT, "answer", analyzer)
    query = parser.parse(QueryParser.escape(answerA))
    result = searcher.search(query, 1)

    scoreDocs = result.scoreDocs
    #for scoreDoc in scoreDocs:
    #    doc = searcher.doc(scoreDoc.doc)
    #    print answerA, doc.get("answer"), scoreDoc.score

    if len(scoreDocs) > 0:
        return result.getMaxScore()
    else:
        return 0

print score_answer("at the tissue level", "at the system level")
print score_answer("at the organ level", "at the system level")
print score_answer("at the system level", "at the system level")
print score_answer("at the cellular level", "at the system level")

0.0281300246716
0.0281300246716
0.271222114563
0.0281300246716


In [9]:
correct = 0
results = []
with open("../data/training_set.tsv") as f:
    csv_reader = csv.reader(f, delimiter="\t", strict=True, quoting=csv.QUOTE_NONE)
    header = next(csv_reader)  # ignore header
    i = 0
    for id, question, right, answerA, answerB, answerC, answerD in list(csv_reader):
        query = parser.parse(QueryParser.escape(question))
        result = searcher.search(query, 100)

        #print question, answerA, answerB, answerC, answerD
        
        scoreA = 0
        scoreB = 0
        scoreC = 0
        scoreD = 0
        scoreDocs = result.scoreDocs
        for scoreDoc in scoreDocs:
            doc = searcher.doc(scoreDoc.doc)
            question = doc.get("question")
            answer = doc.get("answer")
            docscore = scoreDoc.score
            #print question, answer, docscore

            scoreA += docscore * score_answer(answer, answerA)
            scoreB += docscore * score_answer(answer, answerB)
            scoreC += docscore * score_answer(answer, answerC)
            scoreD += docscore * score_answer(answer, answerD)
        
        scores = [scoreA, scoreB, scoreC, scoreD]
        best = np.argmax(scores)
        best = chr(ord('A') + best)
        results.append([id, best] + scores)

        #print scoreA, scoreB, scoreC, scoreD
        #print right, best
        i += 1
        if i % 100 == 0:
            print i

        if best == right:
            correct += 1

accuracy = float(correct) / len(results)
print "Correct: %d Total: %d Accuracy: %f" % (correct, len(results), accuracy)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
Correct: 1238 Total: 2500 Accuracy: 0.495200


In [10]:
with open("lucene100_predictions_training.csv", "w") as f:
    writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for i in xrange(len(results)):
        writer.writerow(results[i])

In [11]:
correct = 0
results = []
with open("../data/test_set.tsv") as f:
    csv_reader = csv.reader(f, delimiter="\t", strict=True, quoting=csv.QUOTE_NONE)
    header = next(csv_reader)  # ignore header
    i = 0
    for id, question, answerA, answerB, answerC, answerD in list(csv_reader):
        query = parser.parse(QueryParser.escape(question))
        result = searcher.search(query, 100)

        #print question, answerA, answerB, answerC, answerD
        
        scoreA = 0
        scoreB = 0
        scoreC = 0
        scoreD = 0
        scoreDocs = result.scoreDocs
        for scoreDoc in scoreDocs:
            doc = searcher.doc(scoreDoc.doc)
            question = doc.get("question")
            answer = doc.get("answer")
            docscore = scoreDoc.score
            #print question, answer, docscore

            scoreA += docscore * score_answer(answer, answerA)
            scoreB += docscore * score_answer(answer, answerB)
            scoreC += docscore * score_answer(answer, answerC)
            scoreD += docscore * score_answer(answer, answerD)
        
        scores = [scoreA, scoreB, scoreC, scoreD]
        best = np.argmax(scores)
        best = chr(ord('A') + best)
        results.append([id, best] + scores)

        #print scoreA, scoreB, scoreC, scoreD
        #print right, best
        i += 1
        if i % 100 == 0:
            print i


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [12]:
with open("lucene100_predictions_test.csv", "w") as f:
    writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for i in xrange(len(results)):
        writer.writerow(results[i])